In [1]:
import pandas as pd

df1 = pd.read_csv("ticketmaster.csv")
df1.head(5)

,line_num,title,createdTime,status,priority,extId,user.identity.name,user.identity.displayName,user.emailId,user.department,extId.1,integrationId,user.identity.name.1,user.identity.displayName.1,user.emailId.1
0,0.0,Scan my computer now,1.574306e+09,Closed,High,stuart.thomson@acme.com,Stuart Thomson,Stuart Thomson,stuart.thomson@acme.com,HR,gearldine_gellinger@gellinger.com,1.0,Gearldine,Gearldine Gellinger,gearldine_gellinger@gellinger.com
1,1.0,how to find address book,1.579539e+09,Closed,Low,peter.macron@acme.com,Peter Macron,Peter Macron,peter.macron@acme.com,Engineering,lnewville@newville.com,1.0,Lenna,Lenna Newville,lnewville@newville.com
2,2.0,Solution 2,1.577330e+09,Closed,Medium,victoria.lewis@acme.com,Victoria Lewis,Victoria Lewis,victoria.lewis@acme.com,Legal,virgina_tegarden@tegarden.com,1.0,Virgina,Virgina Tegarden,virgina_tegarden@tegarden.com
3,3.0,real time scanning wont turn on,1.575271e+09,Closed,Low,vincent.lee@acme.com,Vincent Lee,Vincent Lee,vincent.lee@acme.com,Culture,tfrankel@aol.com,1.0,Tiera,Tiera Frankel,tfrankel@aol.com
4,4.0,Refund,1.588172e+09,Closed,Low,kim.anderson@acme.com,Kim Anderson,Kim Anderson,kim.anderson@acme.com,Product & Services,norah.waymire@gmail.com,1.0,Norah,Norah Waymire,norah.waymire@gmail.com


In [3]:
df1.columns

Index(['line_num', 'title', 'createdTime', 'status', 'priority', 'extId',
       'user.identity.name', 'user.identity.displayName', 'user.emailId',
       'user.department', 'extId.1', 'integrationId', 'user.identity.name.1',
       'user.identity.displayName.1', 'user.emailId.1'],
      dtype='object')

In [124]:
# Connect and create database

import mysql.connector

mydb = mysql.connector.connect(
  host="localhost", port = "3306",
  user="root", password = "Ryan20@gtown"
)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE loopsdb2")

In [125]:
# Connect to database
mydb = mysql.connector.connect(
  host="localhost", port = "3306",
  user="root", password = "Ryan20@gtown", database = "loopsdb2"
)

In [126]:
# Create table

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="Ryan20@gtown",
  database="loopsdb2"
)

mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE ticketmaster9 (id INT AUTO_INCREMENT PRIMARY KEY, title LONGTEXT, sentiment VARCHAR(255), product_entities LONGTEXT)")

In [128]:
# Add titles into table

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="Ryan20@gtown",
  database="loopsdb2"
)

mycursor = mydb.cursor()

sql = "INSERT INTO ticketmaster9 (title) VALUES (%s)"
val = []
for i in df1.index:   
    val.append(str(df1['title'][i]))
val = [x for x in zip(*[iter(val)])]
mycursor.executemany(sql, val)
mydb.commit()

In [129]:
# Access data from table

import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="Ryan20@gtown",
  database="loopsdb2"
)

mycursor = mydb.cursor()
sql = "SELECT title FROM ticketmaster9"
mycursor.execute(sql)

myresult = mycursor.fetchall()

In [130]:
# Check data in table

len(myresult)

9964

In [131]:
myresult[1:5]

[('how to find address book',),
 ('Solution 2',),
 ('real time scanning wont turn on',),
 ('Refund',)]

In [132]:
df1['title'][1:5]

1           how to find address book
2                         Solution 2
3    real time scanning wont turn on
4                             Refund
Name: title, dtype: object

In [121]:
# Check maximum number of tokens, since BERT/RoBERTa model only considers sequences
#  lengths of up to 512 tokens (uses truncation if needed) 
lengths=[]
for i in range(0, len(df_title_list)-1):
    #print(df_title_list[i])
    lengths.append(len(str(df_title_list[i]).split()))

max(lengths)

78

1. Classify titles into sentiment categories using pretrained RoBERTa model from Sentence Transformers package


In [119]:
# Get BERT embeddings for sentiments and titles

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('stsb-roberta-large')
model.max_seq_length


  0%|          | 0.00/1.31G [00:00<?, ?B/s]
  0%|          | 32.8k/1.31G [00:00<1:52:14, 195kB/s]
  0%|          | 49.2k/1.31G [00:00<2:26:41, 149kB/s]
  0%|          | 98.3k/1.31G [00:00<2:05:44, 174kB/s]
  0%|          | 180k/1.31G [00:00<1:42:13, 214kB/s] 
  0%|          | 279k/1.31G [00:00<1:19:10, 277kB/s]
  0%|          | 346k/1.31G [00:00<1:05:08, 336kB/s]
  0%|          | 426k/1.31G [00:01<57:37, 380kB/s]  
  0%|          | 573k/1.31G [00:01<45:05, 485kB/s]
  0%|          | 651k/1.31G [00:01<40:09, 545kB/s]
  0%|          | 803k/1.31G [00:01<34:21, 637kB/s]
  0%|          | 999k/1.31G [00:01<27:34, 794kB/s]
  0%|          | 1.12M/1.31G [00:01<26:10, 836kB/s]
  0%|          | 1.36M/1.31G [00:01<21:38, 1.01MB/s]
  0%|          | 1.57M/1.31G [00:01<18:37, 1.17MB/s]
  0%|          | 1.77M/1.31G [00:01<17:13, 1.27MB/s]
  0%|          | 2.08M/1.31G [00:02<14:19, 1.53MB/s]
  0%|          | 2.28M/1.31G [00:02<13:48, 1.58MB/s]
  0%|          | 2.57M/1.31G [00:02<11:58, 1.83MB/s]
  0%| 

  4%|▍         | 54.9M/1.31G [00:17<05:47, 3.62MB/s]
  4%|▍         | 55.3M/1.31G [00:17<08:27, 2.48MB/s]
  4%|▍         | 55.7M/1.31G [00:17<07:26, 2.82MB/s]
  4%|▍         | 56.1M/1.31G [00:17<06:39, 3.15MB/s]
  4%|▍         | 56.5M/1.31G [00:17<06:54, 3.03MB/s]
  4%|▍         | 56.9M/1.31G [00:17<06:28, 3.23MB/s]
  4%|▍         | 57.2M/1.31G [00:17<06:19, 3.32MB/s]
  4%|▍         | 57.6M/1.31G [00:18<06:10, 3.39MB/s]
  4%|▍         | 58.0M/1.31G [00:18<06:37, 3.16MB/s]
  4%|▍         | 58.3M/1.31G [00:18<06:27, 3.24MB/s]
  4%|▍         | 58.7M/1.31G [00:18<06:01, 3.47MB/s]
  5%|▍         | 59.1M/1.31G [00:18<05:51, 3.57MB/s]
  5%|▍         | 59.5M/1.31G [00:18<05:50, 3.58MB/s]
  5%|▍         | 59.9M/1.31G [00:18<05:53, 3.55MB/s]
  5%|▍         | 60.3M/1.31G [00:18<05:35, 3.73MB/s]
  5%|▍         | 60.7M/1.31G [00:18<05:49, 3.59MB/s]
  5%|▍         | 61.1M/1.31G [00:19<05:37, 3.71MB/s]
  5%|▍         | 61.5M/1.31G [00:19<05:39, 3.68MB/s]
  5%|▍         | 61.9M/1.31G [00:19<05:31, 3.7

  9%|▉         | 117M/1.31G [00:32<04:41, 4.26MB/s]
  9%|▉         | 117M/1.31G [00:32<04:49, 4.13MB/s]
  9%|▉         | 117M/1.31G [00:32<04:50, 4.12MB/s]
  9%|▉         | 118M/1.31G [00:33<04:39, 4.29MB/s]
  9%|▉         | 118M/1.31G [00:33<04:42, 4.24MB/s]
  9%|▉         | 119M/1.31G [00:33<05:10, 3.85MB/s]
  9%|▉         | 119M/1.31G [00:33<05:02, 3.94MB/s]
  9%|▉         | 120M/1.31G [00:33<05:03, 3.93MB/s]
  9%|▉         | 120M/1.31G [00:33<04:52, 4.08MB/s]
  9%|▉         | 121M/1.31G [00:33<04:53, 4.07MB/s]
  9%|▉         | 121M/1.31G [00:33<04:47, 4.15MB/s]
  9%|▉         | 121M/1.31G [00:33<04:47, 4.15MB/s]
  9%|▉         | 122M/1.31G [00:33<04:40, 4.25MB/s]
  9%|▉         | 122M/1.31G [00:34<04:40, 4.24MB/s]
  9%|▉         | 123M/1.31G [00:34<04:53, 4.05MB/s]
  9%|▉         | 123M/1.31G [00:34<04:47, 4.15MB/s]
  9%|▉         | 124M/1.31G [00:34<04:41, 4.23MB/s]
  9%|▉         | 124M/1.31G [00:34<04:48, 4.13MB/s]
  9%|▉         | 124M/1.31G [00:34<04:47, 4.14MB/s]
  9%|▉      

 14%|█▎        | 179M/1.31G [00:48<05:16, 3.58MB/s]
 14%|█▎        | 180M/1.31G [00:48<05:04, 3.72MB/s]
 14%|█▎        | 180M/1.31G [00:48<04:50, 3.90MB/s]
 14%|█▎        | 180M/1.31G [00:48<04:55, 3.83MB/s]
 14%|█▍        | 181M/1.31G [00:48<04:47, 3.94MB/s]
 14%|█▍        | 181M/1.31G [00:49<05:08, 3.67MB/s]
 14%|█▍        | 182M/1.31G [00:49<05:14, 3.60MB/s]
 14%|█▍        | 182M/1.31G [00:49<05:04, 3.72MB/s]
 14%|█▍        | 182M/1.31G [00:49<05:23, 3.50MB/s]
 14%|█▍        | 183M/1.31G [00:49<04:58, 3.79MB/s]
 14%|█▍        | 183M/1.31G [00:49<05:03, 3.72MB/s]
 14%|█▍        | 184M/1.31G [00:49<05:24, 3.49MB/s]
 14%|█▍        | 184M/1.31G [00:49<06:30, 2.89MB/s]
 14%|█▍        | 184M/1.31G [00:50<07:30, 2.51MB/s]
 14%|█▍        | 185M/1.31G [00:50<07:12, 2.61MB/s]
 14%|█▍        | 185M/1.31G [00:50<06:26, 2.92MB/s]
 14%|█▍        | 186M/1.31G [00:50<05:36, 3.35MB/s]
 14%|█▍        | 186M/1.31G [00:50<05:23, 3.49MB/s]
 14%|█▍        | 186M/1.31G [00:50<05:10, 3.63MB/s]
 14%|█▍     

 20%|██        | 266M/1.31G [01:09<02:35, 6.75MB/s]
 20%|██        | 267M/1.31G [01:09<02:37, 6.65MB/s]
 20%|██        | 267M/1.31G [01:10<02:33, 6.83MB/s]
 20%|██        | 268M/1.31G [01:10<02:30, 6.96MB/s]
 20%|██        | 269M/1.31G [01:10<02:32, 6.85MB/s]
 21%|██        | 270M/1.31G [01:10<02:31, 6.90MB/s]
 21%|██        | 270M/1.31G [01:10<02:40, 6.51MB/s]
 21%|██        | 271M/1.31G [01:10<02:34, 6.75MB/s]
 21%|██        | 272M/1.31G [01:10<02:34, 6.75MB/s]
 21%|██        | 272M/1.31G [01:10<02:33, 6.77MB/s]
 21%|██        | 273M/1.31G [01:10<02:30, 6.94MB/s]
 21%|██        | 274M/1.31G [01:10<02:28, 7.01MB/s]
 21%|██        | 275M/1.31G [01:11<02:37, 6.59MB/s]
 21%|██        | 275M/1.31G [01:11<02:34, 6.72MB/s]
 21%|██        | 276M/1.31G [01:11<02:36, 6.64MB/s]
 21%|██        | 277M/1.31G [01:11<02:31, 6.86MB/s]
 21%|██        | 277M/1.31G [01:11<02:36, 6.63MB/s]
 21%|██        | 278M/1.31G [01:11<02:32, 6.78MB/s]
 21%|██        | 279M/1.31G [01:11<02:40, 6.45MB/s]
 21%|██▏    

...

 82%|████████▏ | 1.07G/1.31G [03:17<00:34, 7.07MB/s]
 82%|████████▏ | 1.07G/1.31G [03:17<00:33, 7.23MB/s]
 82%|████████▏ | 1.07G/1.31G [03:17<00:33, 7.22MB/s]
 82%|████████▏ | 1.07G/1.31G [03:17<00:34, 7.00MB/s]
 82%|████████▏ | 1.07G/1.31G [03:17<00:32, 7.28MB/s]
 82%|████████▏ | 1.08G/1.31G [03:17<00:32, 7.35MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:31, 7.42MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:49, 4.77MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:45, 5.22MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:41, 5.61MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:38, 6.11MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:36, 6.34MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:35, 6.60MB/s]
 82%|████████▏ | 1.08G/1.31G [03:18<00:33, 6.89MB/s]
 82%|████████▏ | 1.08G/1.31G [03:19<00:33, 6.88MB/s]
 82%|████████▏ | 1.08G/1.31G [03:19<00:31, 7.39MB/s]
 82%|████████▏ | 1.08G/1.31G [03:19<00:45, 5.08MB/s]
 83%|████████▎ | 1.08G/1.31G [03:19<00:41, 5.58MB/s]
 83%|████████▎ | 1.09G/1.31G [03:19<00:38, 6.0

 90%|████████▉ | 1.18G/1.31G [03:34<00:41, 3.30MB/s]
 90%|████████▉ | 1.18G/1.31G [03:34<00:38, 3.53MB/s]
 90%|████████▉ | 1.18G/1.31G [03:34<00:36, 3.74MB/s]
 90%|████████▉ | 1.18G/1.31G [03:34<00:38, 3.57MB/s]
 90%|████████▉ | 1.18G/1.31G [03:34<00:39, 3.39MB/s]
 90%|████████▉ | 1.18G/1.31G [03:34<00:39, 3.41MB/s]
 90%|████████▉ | 1.18G/1.31G [03:34<00:37, 3.60MB/s]
 90%|████████▉ | 1.18G/1.31G [03:35<00:34, 3.89MB/s]
 90%|████████▉ | 1.18G/1.31G [03:35<00:33, 3.99MB/s]
 90%|████████▉ | 1.18G/1.31G [03:35<00:33, 3.98MB/s]
 90%|████████▉ | 1.18G/1.31G [03:35<00:31, 4.18MB/s]
 90%|████████▉ | 1.18G/1.31G [03:35<00:35, 3.77MB/s]
 90%|████████▉ | 1.18G/1.31G [03:35<00:33, 3.97MB/s]
 90%|████████▉ | 1.18G/1.31G [03:35<00:33, 3.88MB/s]
 90%|█████████ | 1.18G/1.31G [03:35<00:34, 3.85MB/s]
 90%|█████████ | 1.18G/1.31G [03:35<00:36, 3.57MB/s]
 90%|█████████ | 1.18G/1.31G [03:36<00:34, 3.82MB/s]
 90%|█████████ | 1.18G/1.31G [03:36<00:32, 3.97MB/s]
 90%|█████████ | 1.18G/1.31G [03:36<00:31, 4.1

 94%|█████████▍| 1.24G/1.31G [03:49<00:17, 4.12MB/s]
 94%|█████████▍| 1.24G/1.31G [03:50<00:17, 4.10MB/s]
 94%|█████████▍| 1.24G/1.31G [03:50<00:17, 4.24MB/s]
 95%|█████████▍| 1.24G/1.31G [03:50<00:17, 4.19MB/s]
 95%|█████████▍| 1.24G/1.31G [03:50<00:16, 4.21MB/s]
 95%|█████████▍| 1.24G/1.31G [03:50<00:18, 3.84MB/s]
 95%|█████████▍| 1.24G/1.31G [03:50<00:16, 4.25MB/s]
 95%|█████████▍| 1.24G/1.31G [03:50<00:16, 4.21MB/s]
 95%|█████████▍| 1.24G/1.31G [03:50<00:16, 4.11MB/s]
 95%|█████████▍| 1.24G/1.31G [03:50<00:16, 4.21MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:15, 4.38MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:16, 4.25MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:15, 4.26MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:16, 4.02MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:17, 3.93MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:17, 3.76MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:17, 3.89MB/s]
 95%|█████████▍| 1.25G/1.31G [03:51<00:16, 3.98MB/s]
 95%|█████████▌| 1.25G/1.31G [03:51<00:17, 3.8

 99%|█████████▉| 1.31G/1.31G [04:05<00:02, 3.37MB/s]
 99%|█████████▉| 1.31G/1.31G [04:05<00:02, 3.36MB/s]
 99%|█████████▉| 1.31G/1.31G [04:05<00:02, 3.40MB/s]
 99%|█████████▉| 1.31G/1.31G [04:06<00:02, 3.46MB/s]
 99%|█████████▉| 1.31G/1.31G [04:06<00:02, 3.45MB/s]
 99%|█████████▉| 1.31G/1.31G [04:06<00:02, 3.28MB/s]
100%|█████████▉| 1.31G/1.31G [04:06<00:01, 3.29MB/s]
100%|█████████▉| 1.31G/1.31G [04:06<00:01, 3.54MB/s]
100%|█████████▉| 1.31G/1.31G [04:06<00:01, 3.73MB/s]
100%|█████████▉| 1.31G/1.31G [04:06<00:01, 3.53MB/s]
100%|█████████▉| 1.31G/1.31G [04:06<00:01, 3.24MB/s]
100%|█████████▉| 1.31G/1.31G [04:06<00:01, 3.39MB/s]
100%|█████████▉| 1.31G/1.31G [04:07<00:01, 3.70MB/s]
100%|█████████▉| 1.31G/1.31G [04:07<00:00, 3.69MB/s]
100%|█████████▉| 1.31G/1.31G [04:07<00:00, 3.62MB/s]
100%|█████████▉| 1.31G/1.31G [04:07<00:00, 3.44MB/s]
100%|█████████▉| 1.31G/1.31G [04:07<00:00, 3.60MB/s]
100%|█████████▉| 1.31G/1.31G [04:07<00:00, 3.68MB/s]
100%|█████████▉| 1.31G/1.31G [04:07<00:00, 3.6

128

In [133]:
sentences = ['Need Help Understanding', 'Application Error', 'New Feature Request', 'Terminate Service']

for i in range(0, len(myresult) - 1):
    sentences.append(myresult[i][0])
    
sentence_embeddings = model.encode(sentences)



In [134]:
# Assign sentiment based on which has the closest cosine similarity to each title

from sentence_transformers import util
import torch
import numpy as np
cosine_matrix_test1 = np.empty((0, 4), int)

# First calculate all cosine similarities
for title_ind in range(0, len(myresult) - 1):
    curr_row = []
    for sentiment_ind in range(0, 4):
        curr_similarity = util.pytorch_cos_sim(sentence_embeddings[title_ind + 4], sentence_embeddings[sentiment_ind])
        curr_row.append(curr_similarity)
    curr_row_final = np.array(curr_row)
    cosine_matrix_test1 = np.vstack((cosine_matrix_test1, curr_row_final))


# Assign highest cosine similarity
indices_top_sentiment =[]
for row in cosine_matrix_test1:
    indices_top_sentiment.append(np.argmax(row))
            



In [135]:
print(indices_top_sentiment)

[1, 0, 3, 1, 1, 1, 0, 1, 1, 1, 3, 1, 1, 2, 2, 3, 3, 1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 3, 2, 1, 3, 3, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 1, 2, 3, 0, 1, 3, 1, 1, 3, 2, 2, 3, 1, 3, 2, 1, 2, 2, 3, 1, 2, 3, 0, 1, 3, 0, 2, 1, 2, 3, 3, 1, 1, 1, 1, 1, 3, 1, 0, 3, 1, 2, 1, 1, 1, 1, 3, 0, 3, 3, 3, 3, 1, 3, 1, 1, 1, 0, 3, 0, 2, 2, 1, 3, 1, 2, 1, 3, 3, 0, 3, 2, 3, 1, 0, 3, 3, 1, 0, 2, 1, 1, 1, 2, 0, 1, 2, 1, 1, 3, 2, 3, 3, 1, 1, 2, 1, 2, 1, 1, 3, 2, 1, 3, 1, 0, 3, 1, 0, 2, 3, 3, 1, 2, 1, 3, 1, 1, 3, 1, 1, 1, 0, 1, 1, 3, 2, 3, 2, 3, 0, 0, 3, 2, 2, 1, 3, 3, 3, 1, 3, 1, 2, 1, 2, 1, 3, 1, 1, 1, 3, 1, 3, 3, 1, 0, 1, 3, 1, 1, 3, 1, 3, 1, 3, 0, 1, 3, 1, 2, 3, 3, 0, 2, 1, 1, 1, 2, 1, 3, 3, 1, 3, 1, 1, 3, 0, 1, 1, 3, 0, 1, 2, 1, 1, 3, 1, 3, 1, 2, 3, 3, 3, 3, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 0, 3, 3, 3, 3, 1, 1, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 1, 0, 1, 2, 0, 1, 1, 3, 3, 2, 3, 2, 1, 2, 2, 1, 2, 1, 2, 2, 3, 1, 1, 1, 1, 3, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 3, 3, 1, 1, 0, 2, 

In [92]:
# Insert sentiments into database table

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="Ryan20@gtown",
  database="loopsdb2"
)

mycursor = mydb.cursor()

sql = "INSERT INTO ticketmaster9 (sentiment) VALUES (%s)"
val = []
for i in range(0, len(indices_top_sentiment) - 1):   
    val.append(str(sentences[indices_top_sentiment[i]]))
val = [x for x in zip(*[iter(val)])]
mycursor.executemany(sql, val)
mydb.commit()

2. Find Product Entities using pretrained model from Flair package

-Categories: Person, Norp, Fac, Org, Gpe, Loc, Product, Event, Work_Of_Art, Law, Language, Date, Time, Percent, Money, Quantity, Ordinal, Cardinal
-Model: word embeddings (bidrectional character language model) inputted into bidirectional lstm 
-Data: telephone conversations, newswire, newsgroups, broadcast news, broadcast conversation, weblogs

In [96]:
from flair.models import SequenceTagger
model = SequenceTagger.load('ner-ontonotes-fast') 

from flair.data import Sentence
pos = []
for i in range(0, len(myresult) - 1):
    sentence_curr = Sentence(myresult[i][0])
    model.predict(sentence_curr)
    pos.append(sentence_curr.to_tagged_string())



2021-03-22 22:19:43,246 https://nlp.informatik.hu-berlin.de/resources/models/ner-ontonotes-fast/en-ner-ontonotes-fast-v0.4.pt not found in cache, downloading to /var/folders/0v/sp41zrrd26z8k1vwlt7jw2mm0000gn/T/tmpt_te5uac



  3%|▎         | 37225472/1331379415 [00:13<05:16, 4093466.29B/s]


  6%|▌         | 82017280/1331379415 [00:26<05:07, 4059946.48B/s]


 10%|█         | 134077440/1331379415 [00:38<05:36, 3557190.56B/s]


 14%|█▎        | 182060032/1331379415 [00:51<05:31, 3463494.74B/s]


...

 94%|█████████▎| 1247175680/1331379415 [05:13<00:18, 4469710.35B/s]


 98%|█████████▊| 1302553600/1331379415 [05:26<00:06, 4521197.95B/s]


100%|██████████| 1331379415/1331379415 [05:32<00:00, 4002227.70B/s]

2021-03-22 22:25:16,496 copying /var/folders/0v/sp41zrrd26z8k1vwlt7jw2mm0000gn/T/tmpt_te5uac to cache at /Users/ardeshir/.flair/models/en-ner-ontonotes-fast-v0.4.pt


2021-03-22 22:25:20,337 removing temp file /var/folders/0v/sp41zrrd26z8k1vwlt7jw2mm0000gn/T/tmpt_te5uac
2021-03-22 22:25:20,670 loading file /Users/ardeshir/.flair/models/en-ner-ontonotes-fast-v0.4.pt


In [97]:
pos

['Scan my computer now',
 'how to find address book',
 'Solution 2 <S-CARDINAL>',
 'real time scanning wont turn on',
 'Refund',
 '800 number customer suport',
 'how to update acme',
 'how do i reset true key password',
 'Turn on Fire Wall ?',
 '"" You tell me I have a Trojan <S-PRODUCT> ( after a scan three <B-DATE> days <E-DATE> ago,Customers,Proof Of Concept,Lindsay Moore R0GrzG0BF9wAg _ bpi2be,uberdoc _ index _ requests , , faq,devops , , Slack,false,4',
 'cancel my renewal',
 'i cannot renew my suscription',
 'Why is webadvisor not working ?',
 'i have mcfee got new brain . dont have mcfee now . just ordered again',
 'How do I install MC Afee Secure',
 'my email account no longer exists',
 'cancel subscription',
 'firewall missing files',
 'I have given my payment information and am trying to install the protection but something went wrong . Do I have protection ?',
 'i have error in installing',
 'agent',
 'i have auto subscription renewal due on 29 <B-DATE> / <I-DATE> 10 <I-DATE

In [ ]:
# Insert titles with entities embedded into database table

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="Ryan20@gtown",
  database="loopsdb2"
)

mycursor = mydb.cursor()

sql = "INSERT INTO ticketmaster9 (product_entities) VALUES (%s)"
val = []
for i in range(0, len(pos) - 1):   
    val.append(str(pos))
val = [x for x in zip(*[iter(val)])]
mycursor.executemany(sql, val)
mydb.commit()